# $DBSCAN$ and Plotting

Author: `Márcio Lopes Jr` 

*Master's student of `Computer Engineering, Intelligent Information Processing` at UFRN-Natal*.

## Libraries

In [2]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, Normalizer, PowerTransformer
import gc

# Special file for plots
from codes import *

## Files

In [4]:
# Info pastas
folder_name = 'files'

# Fontw Gill Sans
gs_font = fm.FontProperties(fname=f'{folder_name}/Gill Sans.otf')

# Cluster data and image folder
path_cities_classes = f"{folder_name}/km_cities_clusters.csv"
path_cluster_centres_dif = f"{folder_name}/km_cluster_centres_dif.csv"
path_cluster_centres_abs = f"{folder_name}/km_cluster_centres_abs.csv"
path_plots = f'{folder_name}/imagens'

# Load cluster data
centros_d = pd.read_csv(path_cluster_centres_dif, index_col=0).reset_index(drop=True)
centros_t = pd.read_csv(path_cluster_centres_abs, index_col=0, header=None).reset_index(drop=True)
centros_t.columns = centros_d.columns

## Correlation+PCA Reduction

In [13]:
# Creates correlation similarity matrix
centros_pos = centros_d.iloc[:, :-3].copy()
centros_pos[:] = MinMaxScaler().fit_transform(centros_pos)
centros_corr = centros_pos.transpose().corr()
centros_x_corr_copy = centros_corr.copy()
# Orders matrix by similarity
sorted_corr = cluster_corr(centros_corr)

In [17]:
PCA_CORR = PCA(0.99)
pca_matrix = pd.DataFrame(PCA_CORR.fit_transform(centros_x_corr_copy))
pca_matrix.columns = [f'x{i}' for i in range(len(pca_matrix.columns))]
pca_matrix.head()

,x0,x1,x2,x3
0,-10.971114,0.653634,-1.232504,-0.838643
1,-11.030938,0.818901,-1.311083,-0.667198
2,-11.030938,0.818901,-1.311083,-0.667198
3,-10.856674,0.457702,-1.184021,-0.995153
4,-11.030938,0.818901,-1.311083,-0.667198


## PCA Validation of $k$-Means

In [18]:
dbs = DBSCAN(eps=.06, min_samples=50, metric='euclidean')
dbs.fit(MinMaxScaler().fit_transform(pca_matrix))

clusters = np.unique(dbs.labels_)
np.sort(clusters)

pd.Series(dbs.labels_).value_counts()

-1    336
 0    240
 1    201
 2    167
 3    135
 4    135
 6     80
 5     43
dtype: int64

In [19]:
centros_d = pd.read_csv(path_cluster_centres_dif, index_col=0).reset_index(drop=True)
centros_t = pd.read_csv(path_cluster_centres_abs, index_col=0, header=None).reset_index(drop=True)
centros_t.columns = centros_d.columns

centros_t['cluster_principal'] = dbs.labels_
centros_t.loc[centros_t.cluster_principal == -1, 'cluster_principal'] = np.nan

centros_d['cluster_principal'] = dbs.labels_
centros_d.loc[centros_d.cluster_principal == -1, 'cluster_principal'] = np.nan

In [20]:
centros_t.drop(columns=centros_t.loc[:,(centros_t.mean() > 1).values].columns[:-4])\
.groupby('cluster_principal').std().mean(axis=1)

cluster_principal
0.0    0.188830
1.0    0.160629
2.0    0.158978
3.0    0.159312
4.0    0.151271
5.0    0.154251
6.0    0.149504
dtype: float64